In [7]:
!pip install kaggle
!mkdir -p ~/.kaggle
from google.colab import files
uploaded=files.upload()
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Saving movie_review.csv to movie_review.csv
mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [8]:
!kaggle datasets download -d nltkdata/movie-review

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [3]:
!unzip /content/movie-review.zip

unzip:  cannot find or open /content/movie-review.zip, /content/movie-review.zip.zip or /content/movie-review.zip.ZIP.


In [9]:
df=pd.read_csv('/content/movie_review.csv')
df

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos
...,...,...,...,...,...,...
64715,9,cv999,14636,20,that lack of inspiration can be traced back to...,neg
64716,9,cv999,14636,21,like too many of the skits on the current inca...,neg
64717,9,cv999,14636,22,"after watching one of the "" roxbury "" skits on...",neg
64718,9,cv999,14636,23,"bump unsuspecting women , and . . . that's all .",neg


In [10]:
import string
# Télécharger la liste des stop words (si ce n'est pas déjà fait)
nltk.download('stopwords')

# Fonction pour effectuer le prétraitement du texte
def preprocess_text(text):
    # Supprimer la ponctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    # Convertir en minuscules
    text = text.lower()
    # Supprimer les stop words
    stop_words = set(stopwords.words('english'))
    tokens = text.split()
    text = ' '.join([word for word in tokens if word not in stop_words])
    return text

# Appliquer la fonction de prétraitement à la colonne "Text" de votre DataFrame
df['text'] = df['text'].apply(preprocess_text)
#vecteur par sentence
df
# Maintenant, la colonne "Text" de votre DataFrame contient les données textuelles prétraitées

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted comic books plenty success wheth...,pos
1,0,cv000,29590,1,starters created alan moore eddie campbell bro...,pos
2,0,cv000,29590,2,say moore campbell thoroughly researched subje...,pos
3,0,cv000,29590,3,book graphic novel 500 pages long includes nea...,pos
4,0,cv000,29590,4,words dont dismiss film source,pos
...,...,...,...,...,...,...
64715,9,cv999,14636,20,lack inspiration traced back insipid characters,neg
64716,9,cv999,14636,21,like many skits current incarnation saturdayni...,neg
64717,9,cv999,14636,22,watching one roxbury skits snl come away chara...,neg
64718,9,cv999,14636,23,bump unsuspecting women thats,neg


In [11]:
from gensim.models import Word2Vec

# Votre DataFrame 'df' contenant les colonnes 'text' et 'tag'
# Assurez-vous que le prétraitement a déjà été appliqué à la colonne 'text'

# Obtenez une liste de listes de mots à partir de la colonne 'text'
preprocessed_data = df['text'].apply(lambda x: x.split())

# Entraîner le modèle Word2Vec
model = Word2Vec(sentences=preprocessed_data, vector_size=100, window=5, min_count=1, sg=0)

# Arguments :
# - sentences : Les données textuelles prétraitées.
# - vector_size : La dimension des vecteurs de mots (taille des embeddings).
# - window : La taille de la fenêtre de contexte pour l'apprentissage.
# - min_count : Le nombre minimum d'occurrences d'un mot pour être pris en compte.
# - sg : Utiliser Skip-gram (1) ou CBOW (0).

# Sauvegarder le modèle
model.save("votre_modele_word2vec.model")


In [12]:
import numpy as np
from gensim.models import Word2Vec

# Charger le modèle Word2Vec entraîné
model = Word2Vec.load("votre_modele_word2vec.model")

# Votre DataFrame 'df' contenant les colonnes 'text' et 'tag'
# Assurez-vous que le prétraitement a déjà été appliqué à la colonne 'text'

# Fonction pour obtenir le vecteur moyen d'une revue
def get_review_vector(review, model):
    words = review.split()
    word_vectors = []
    for word in words:
        if word in model.wv:
            word_vectors.append(model.wv[word])
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)  # Vecteur nul si aucun mot trouvé

# Appliquer la fonction pour obtenir les vecteurs de toutes les revues
df['review_vector'] = df['text'].apply(lambda x: get_review_vector(x, model))

# Maintenant, chaque revue est représentée par un vecteur dans la colonne 'review_vector'


In [13]:
from sklearn.model_selection import train_test_split

# Votre DataFrame 'df' contenant les données
# 'X' contient les vecteurs des revues, 'y' contient les étiquettes ('pos' ou 'neg')

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df['review_vector'].tolist(), df['tag'], test_size=0.2, random_state=42)

# 'X_train' et 'X_test' contiennent les vecteurs des revues pour les ensembles d'entraînement et de test
# 'y_train' et 'y_test' contiennent les étiquettes correspondantes

# Vous pouvez ajuster le paramètre 'test_size' pour définir la proportion de données à utiliser pour les tests
# 'random_state' permet de garantir la reproductibilité de la division (utilisez une valeur fixe)


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Utilisez le modèle de régression logistique (Logistic Regression) comme exemple.
# Vous pouvez utiliser le modèle de votre choix en remplaçant LogisticRegression par RandomForestClassifier ou SVC.

# Créer un modèle
model = LogisticRegression()

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
y_pred = model.predict(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Calculer l'accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy : {accuracy:.2f}")

# Calculer la précision
precision = precision_score(y_test, y_pred, average='weighted')  # weighted pour la moyenne pondérée
print(f"Précision : {precision:.2f}")

# Calculer le recall
recall = recall_score(y_test, y_pred, average='weighted')  # weighted pour la moyenne pondérée
print(f"Recall : {recall:.2f}")

# Calculer le F1-score
f1 = f1_score(y_test, y_pred, average='weighted')  # weighted pour la moyenne pondérée
print(f"F1-Score : {f1:.2f}")

# Afficher un rapport de classification détaillé
classification_rep = classification_report(y_test, y_pred)
print("Rapport de classification :\n", classification_rep)

# Afficher la matrice de confusion
confusion = confusion_matrix(y_test, y_pred)
print("Matrice de confusion :\n", confusion)


Accuracy : 0.57
Précision : 0.57
Recall : 0.57
F1-Score : 0.56
Rapport de classification :
               precision    recall  f1-score   support

         neg       0.58      0.46      0.51      6371
         pos       0.56      0.68      0.62      6573

    accuracy                           0.57     12944
   macro avg       0.57      0.57      0.56     12944
weighted avg       0.57      0.57      0.56     12944

Matrice de confusion :
 [[2900 3471]
 [2091 4482]]
